<a href="https://colab.research.google.com/github/quintusvw/colab/blob/GNN/Introduction_to_GNNs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip isntall numpy
!pip isntall tensorflow
!pip install spektral

import numpy as np
import tensorflow as tf
import spektral

ERROR: unknown command "isntall" - maybe you meant "install"
ERROR: unknown command "isntall" - maybe you meant "install"
     |████████████████████████████████| 123 kB 4.0 MB/s 


In [3]:
from spektral.datasets.citation import Cora
dataset = Cora()
graph = dataset[0]
adj, features, labels = graph.a, graph.x, graph.y
train_mask, val_mask, test_mask = dataset.mask_tr, dataset.mask_va, dataset.mask_te

#adj, features, labels, train_mask, val_mask, test_mask = spektral.datasets.citation.Citation('Cora')

#features = features.todense()
adj =  adj.todense() + np.eye(adj.shape[0])
features = features.astype('float32')
adj = adj.astype('float32')

print(features.shape)
print(adj.shape)
print(labels.shape)

print(np.sum(train_mask))
print(np.sum(val_mask))
print(np.sum(test_mask))



(2708, 1433)
(2708, 2708)
(2708, 7)
140
500
1000


/usr/local/lib/python3.7/dist-packages/scipy/sparse/_index.py:126: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  self._set_arrayXarray(i, j, x)


In [4]:
def masked_softmax_cross_entropy(logits, labels, mask):
  loss = tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=labels)
  mask = tf.cast(mask, dtype=tf.float32)
  mask /= tf.reduce_mean(mask)
  loss += mask
  return tf.reduce_mean(loss)

def masked_accuracy(logits, labels, mask):
  correct_prediction = tf.equal(tf.argmax(logits, 1), tf.argmax(labels, 1))
  accuracy_all = tf.cast(correct_prediction, tf.float32)
  mask = tf.cast(mask, dtype=tf.float32)
  mask /= tf.reduce_mean(mask)
  accuracy_all *= mask
  return tf.reduce_mean(accuracy_all)

In [5]:
def gnn(fts, adj, transform, activation):
  seq_fts = transform(fts)
  ret_fts = tf.matmul(adj, seq_fts)
  return activation(ret_fts)

In [6]:
def train_cora(fts, adj, gnn_fn, units, epochs, lr):
  lyr_1 = tf.keras.layers.Dense(units)
  lyr_2 = tf.keras.layers.Dense(7)

  def cora_gnn(fts, adj):
    hidden = gnn_fn(fts, adj, lyr_1, tf.nn.relu)
    logits = gnn_fn(hidden, adj, lyr_2, tf.identity)
    return logits

  optimizer = tf.keras.optimizers.Adam(learning_rate=lr)

  best_accuracy = 0.0
  for ep in range(epochs + 1):
    with tf.GradientTape() as t:
      logits = cora_gnn(fts, adj)
      loss = masked_softmax_cross_entropy(logits, labels, train_mask)

    variables = t.watched_variables()
    grads = t.gradient(loss, variables)
    optimizer.apply_gradients(zip(grads, variables))

    logits = cora_gnn(fts, adj)
    val_accuracy = masked_accuracy(logits, labels, val_mask)
    test_accuracy = masked_accuracy(logits, labels, test_mask)

    if val_accuracy > best_accuracy:
      best_Accuracy = val_accuracy
      print('Epoch', ep, '| Training loss:', loss.numpy(), '| Val accuracy:', 
            val_accuracy.numpy(), '| Test accuracy: ', test_accuracy.numpy())

In [7]:
train_cora(features, adj, gnn, 23, 200, 0.01)

Epoch 0 | Training loss: 6.4603834 | Val accuracy: 0.314 | Test accuracy:  0.322
Epoch 1 | Training loss: 6.0126767 | Val accuracy: 0.103999995 | Test accuracy:  0.141
Epoch 2 | Training loss: 5.049294 | Val accuracy: 0.152 | Test accuracy:  0.175
Epoch 3 | Training loss: 4.4345303 | Val accuracy: 0.24399999 | Test accuracy:  0.271
Epoch 4 | Training loss: 3.4705584 | Val accuracy: 0.524 | Test accuracy:  0.53800005
Epoch 5 | Training loss: 2.7070005 | Val accuracy: 0.542 | Test accuracy:  0.533
Epoch 6 | Training loss: 2.598157 | Val accuracy: 0.532 | Test accuracy:  0.527
Epoch 7 | Training loss: 2.6313097 | Val accuracy: 0.534 | Test accuracy:  0.527
Epoch 8 | Training loss: 2.6126626 | Val accuracy: 0.576 | Test accuracy:  0.562
Epoch 9 | Training loss: 2.4609516 | Val accuracy: 0.672 | Test accuracy:  0.628
Epoch 10 | Training loss: 2.337007 | Val accuracy: 0.68 | Test accuracy:  0.652
Epoch 11 | Training loss: 2.3042836 | Val accuracy: 0.672 | Test accuracy:  0.649
Epoch 12 | Tra

In [9]:
deg = tf.reduce_sum(adj, axis=-1)
train_cora(features, adj / deg, gnn, 32, 200, 0.01)

Epoch 0 | Training loss: 2.9596586 | Val accuracy: 0.37399998 | Test accuracy:  0.389
Epoch 1 | Training loss: 2.8306992 | Val accuracy: 0.408 | Test accuracy:  0.425
Epoch 2 | Training loss: 2.7214448 | Val accuracy: 0.49 | Test accuracy:  0.50399995
Epoch 3 | Training loss: 2.6002038 | Val accuracy: 0.586 | Test accuracy:  0.59000003
Epoch 4 | Training loss: 2.4661965 | Val accuracy: 0.69600004 | Test accuracy:  0.693
Epoch 5 | Training loss: 2.3334837 | Val accuracy: 0.736 | Test accuracy:  0.73399997
Epoch 6 | Training loss: 2.2060604 | Val accuracy: 0.794 | Test accuracy:  0.7849999
Epoch 7 | Training loss: 2.0809884 | Val accuracy: 0.83599997 | Test accuracy:  0.82699984
Epoch 8 | Training loss: 1.9625617 | Val accuracy: 0.86399996 | Test accuracy:  0.8549998
Epoch 9 | Training loss: 1.8566074 | Val accuracy: 0.88 | Test accuracy:  0.8699997
Epoch 10 | Training loss: 1.7634686 | Val accuracy: 0.88399994 | Test accuracy:  0.88299966
Epoch 11 | Training loss: 1.6818161 | Val accura

In [11]:
norm_deg = tf.linalg.diag(1.0 / tf.sqrt(deg))
norm_adj = tf.matmul(norm_deg, tf.matmul(adj, norm_deg))
train_cora(features, norm_adj, gnn, 32, 200, 0.01)

Epoch 0 | Training loss: 2.9631474 | Val accuracy: 0.37399998 | Test accuracy:  0.362
Epoch 1 | Training loss: 2.8212368 | Val accuracy: 0.36599997 | Test accuracy:  0.368
Epoch 2 | Training loss: 2.6930845 | Val accuracy: 0.38599998 | Test accuracy:  0.376
Epoch 3 | Training loss: 2.555481 | Val accuracy: 0.41600004 | Test accuracy:  0.422
Epoch 4 | Training loss: 2.4163578 | Val accuracy: 0.50799996 | Test accuracy:  0.524
Epoch 5 | Training loss: 2.2759051 | Val accuracy: 0.658 | Test accuracy:  0.666
Epoch 6 | Training loss: 2.134493 | Val accuracy: 0.79200006 | Test accuracy:  0.7819999
Epoch 7 | Training loss: 1.9998705 | Val accuracy: 0.85199994 | Test accuracy:  0.8549998
Epoch 8 | Training loss: 1.8800261 | Val accuracy: 0.878 | Test accuracy:  0.88299966
Epoch 9 | Training loss: 1.7745692 | Val accuracy: 0.88600004 | Test accuracy:  0.88799965
Epoch 10 | Training loss: 1.680924 | Val accuracy: 0.878 | Test accuracy:  0.8909997
Epoch 11 | Training loss: 1.599003 | Val accuracy